In [4]:
pip install mysql.connector

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install ahpy


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import mysql.connector
import ahpy

# Connect to the MySQL database
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='electric_car'
)
cursor = connection.cursor()

# Fetch criteria and alternatives from the database
cursor.execute("SELECT DISTINCT criterion1 FROM criteria_comparisons UNION SELECT DISTINCT criterion2 FROM criteria_comparisons")
criteria_rows = cursor.fetchall()
print("Criteria Rows:", criteria_rows)  # Debugging: print criteria rows

criterias = [row[0] for row in criteria_rows]

cursor.execute("SELECT DISTINCT alternative1 FROM alternative_comparisons UNION SELECT DISTINCT alternative2 FROM alternative_comparisons")
alternative_rows = cursor.fetchall()
print("Alternative Rows:", alternative_rows)  # Debugging: print alternative rows

alternatives = [row[0] for row in alternative_rows]

# Fetch criteria comparisons from the database
cursor.execute("SELECT criterion1, criterion2, crit_value FROM criteria_comparisons")
criteria_comparisons = {(row[0], row[1]): row[2] for row in cursor.fetchall()}

# Fetch alternative comparisons from the database
cursor.execute("SELECT criterion, alternative1, alternative2, alt_value FROM alternative_comparisons")
alternative_comparisons = {}
for row in cursor.fetchall():
    criterion, alternative1, alternative2, value = row
    if criterion not in alternative_comparisons:
        alternative_comparisons[criterion] = {}
    alternative_comparisons[criterion][(alternative1, alternative2)] = value

# Close the database connection
connection.close()

# Create the AHP model for criteria
criteria_ahp = ahpy.Compare('Criteria', criteria_comparisons, precision=3)

# Create the AHP models for alternatives with respect to each criterion
alternative_ahp = {}
for criterion in criterias:
    alternative_ahp[criterion] = ahpy.Compare(criterion, alternative_comparisons[criterion], precision=3)

# Combine the criteria and alternative priorities to calculate the overall scores
overall_scores = {}
for alt in alternatives:
    overall_scores[alt] = sum(criteria_ahp.target_weights[crit] * alternative_ahp[crit].target_weights[alt] for crit in criterias)


Criteria Rows: [('harga',), ('kinerja',), ('keamanan',), ('kapasitas baterai',), ('kenyamanan',), ('tingkat penjualan',)]
Alternative Rows: [('Hyundai Ioniq 5 Prime Standard',), ('Wuling Air EV Standard',), ('Toyota BZ4X Panoramic Roof (One Tone Color)',), ('Nissan Leaf One Toned',), ('Lexus UX 300e',), ('Kia EV6 GT Line',), ('Mercedes–Benz EQA 250',), ('BMW iX',), ('Citroen E-C4',), ('Air EV Long',), ('BZ4X Panoramic Roof',), ('Nissan Leaf',), ('Mercedes-Benz EQS 450',), ('BMW i7',), ('Tesla Model Y',), ('Tesla Model Y Long Range',)]


In [7]:
# Insert results into the database
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='electric_car'
)
cursor = connection.cursor()

cursor.execute("DELETE FROM ahp_results")
for idx, (alternative, score) in enumerate(sorted(overall_scores.items(), key=lambda item: item[1], reverse=True), start=1):
    cursor.execute("INSERT INTO ahp_results (id, alternative, score) VALUES (%s, %s, %s)", (idx, alternative, float(score)))

# Commit the transaction
connection.commit()
connection.close()

# Print overall scores for debugging
print("Overall Scores:", overall_scores)

Overall Scores: {'Hyundai Ioniq 5 Prime Standard': 0.06445100000000001, 'Wuling Air EV Standard': 0.076153, 'Toyota BZ4X Panoramic Roof (One Tone Color)': 0.018664999999999998, 'Nissan Leaf One Toned': 0.042092000000000004, 'Lexus UX 300e': 0.036399999999999995, 'Kia EV6 GT Line': 0.030398, 'Mercedes–Benz EQA 250': 0.040816000000000005, 'BMW iX': 0.03096, 'Citroen E-C4': 0.057754999999999994, 'Air EV Long': 0.221266, 'BZ4X Panoramic Roof': 0.050203, 'Nissan Leaf': 0.054644000000000005, 'Mercedes-Benz EQS 450': 0.044923000000000005, 'BMW i7': 0.040892, 'Tesla Model Y': 0.118534, 'Tesla Model Y Long Range': 0.072841}
